In [1]:
import json
from web3 import Web3
import os
import platform
from getpass import getpass
import random

In [95]:
operating_system = platform.system()
d = '/'
if operating_system == 'Windows':
    d = '\\'

ganache_url = "HTTP://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

# This is our main account that controls everything
web3.eth.defaultAccount = web3.eth.accounts[0]

In [96]:
web3.eth.accounts

['0xb6ef75b96D856a6cE0C796be4a7ABfB81E5D600A',
 '0xa2D0B6dbCd04a789da44CCECA9d66B6dfc93A124',
 '0xEE4F5078d802207d68eE9668AE6b82FCFCF7e290',
 '0x9cbFF84198C1c3ed898C52Fbc0076Bf008476C4B',
 '0x18a31E094820e4C9C79a073A0927FA72635b18C7',
 '0xDED00A7f19D445B2c6D3ddFffbf78BB490771937',
 '0x37594b8C8aC1D94457C86e30dE793099CE659817',
 '0x6a1ab8E6bb7e9CC3463997FC7F38e3B3855282b5',
 '0xAfd3bB14ff7735B0D9792e5bcc9BB5e758a32ccB',
 '0x8b23fa43919cB11b9CDa04FA45B92aaA0A0Adcd8']

In [97]:
def getContract(filename):
    with open('..'+ d +'build'+ d +'contracts'+ d +filename) as contractFile:
        contractJson=json.load(contractFile)
        contractABI =contractJson['abi']
        contractAdd=web3.toChecksumAddress(contractJson['networks']['5777']['address']) 
        return contractABI, contractAdd



# Get the organization contract address
organizationContractABI, organizationContractAdd = getContract('Organization.json')	
organizationContract = web3.eth.contract(address=organizationContractAdd, abi=organizationContractABI)

#get the user contract address
userContractABI,userContractAdd = getContract('User.json')
userContract = web3.eth.contract(address=userContractAdd, abi=userContractABI)

# Get the accounts contract address
accountsContractABI,accountsContractAdd = getContract('Accounts.json')
accountsConract = web3.eth.contract(address=accountsContractAdd, abi=accountsContractABI)

loansContractABI, loansContractAddress= getContract('Loans.json')


In [98]:
# This should be changed to be a more robus way. We can give the loansContractAddress using the constructor
# but this has problems when doing this in the 2_deploy_contracts.js
userContract.functions.setLoansContractAddress(loansContractAddress).transact()
organizationContract.functions.setLoansContractAddress(loansContractAddress).transact()

HexBytes('0xc8f7804c692adac3bdeeff147081e514a701862572c6c617aad67a17fc806dcd')

HexBytes('0xc65307278049c77501b46db2385c35c30edb7a37bd15571db9f7a04618c1d3dc')

In [99]:
accountsConract.functions.add(web3.eth.accounts[1], False).transact()
accountsConract.functions.add(web3.eth.accounts[2], False).transact()
accountsConract.functions.add(web3.eth.accounts[5], True).transact()
accountsConract.functions.add(web3.eth.accounts[6], True).transact()

HexBytes('0xd5df8709f2d385f71333b7dbdb8fa5ac5bd3412b0cbb481cdbc2e7b3db050560')

HexBytes('0xa0e4d7b4e0ff04b39a06c6d1433368ae919745ea334300da9bab53579ced2f48')

HexBytes('0xe7d1e5353b27d07d2fb0999afa7b77965c453493fe54b8cdb49869af8ac98c72')

HexBytes('0x759a1e119f5866bcfb48f3f21df2b422a0146bdfd7846d9ecbb6378a11a7dcab')

In [113]:
def deleteUser():
    accountsConract.functions.deleteAccount(web3.eth.accounts[1]).transact()

def createLoan(_loanie, _loaner, _amount, _organizationContract):
    transaction = _organizationContract.functions.createLoan(_loanie, _amount 
    ).buildTransaction({
    'gas': 70000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loaner,
    'nonce': web3.eth.getTransactionCount(_loaner)
    }) 
    _privateKey = getpass("Enter the password of the organization: ")
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=_privateKey)
    print(signed_txn)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    
    return True

In [7]:
deleteUser()

In [101]:
def getPendingLoans(_userContract, _loanieAddress, _privateKey):
    transaction = _userContract.functions.getPendingLoans(
    ).buildTransaction({
    'gas': 70000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loanieAddress,
    'nonce': web3.eth.getTransactionCount(_loanieAddress)
    }) 
    
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=_privateKey)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    receipt = web3.eth.getTransactionReceipt(transaction_hash)
    rich_logs = _userContract.events.getAmounts().processReceipt(receipt)
    event_values = rich_logs[0]['args'] # Dictionary
    return event_values


In [109]:
# Suppose bank wants to create a loan
account = input("Enter the addrses: ")

### Check to see whether it is a user or organization address
index = accountsConract.functions.getIndex(account).call()

Enter the addrses: 0x37594b8C8aC1D94457C86e30dE793099CE659817


In [110]:
index

3

## Get pending loans

In [79]:
# Get pending loans
def getPendingLoansList(_accountIndex, _userContract, _accountsContract, _loanieAddress):
    pendingLoans = [] # 'amount loanerAddress id'
    if _accountIndex != -1:
        loanieType = _accountsContract.functions.getType(_accountIndex).call()
        if not loanieType:
            privateKey = getpass("Enter your password: ")
            values = getPendingLoans(_userContract, _loanieAddress, privateKey)
            for i in range(len(values['_amounts'])):
                string = ''
                for key in values:
                    string += str(values[key][i]) + ' '
                pendingLoans.append(string)
    else:
        print("This account is not registered in our system.")
    return pendingLoans

In [ ]:
getPendingLoansList(index, userContract, accountsConract, account)

## Confirm or reject loans

In [ ]:
# Confirm or reject loans
pendingLoans = getPendingLoansList(index, userContract, accountsConract, account)
private_key = getpass('Enter your loanie password: ')
for pendingLoan in pendingLoans:
    print('for address: ' + pendingLoan.split(' ')[1])
    loanId = pendingLoan.split(' ')[2]
    choice = input('c/r?')
    if choice == 'c':
        transaction = userContract.functions.validateLoan(True,
        int(loanId)).buildTransaction({
        'gas': 70000,
        'gasPrice': web3.toWei('1', 'gwei'),
        'from': account,
        'nonce': web3.eth.getTransactionCount(account)
        }) 
        signed_txn = web3.eth.account.signTransaction(transaction, private_key=private_key)
        transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    elif choice == 'r':
        transaction = userContract.functions.validateLoan(False,
        int(loanId)).buildTransaction({
        'gas': 70000,
        'gasPrice': web3.toWei('1', 'gwei'),
        'from': account,
        'nonce': web3.eth.getTransactionCount(account)
        }) 
        signed_txn = web3.eth.account.signTransaction(transaction, private_key=private_key)
        transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    else:
        print("Not recognized operation.")

# Create a new loan

In [111]:
account

'0x37594b8C8aC1D94457C86e30dE793099CE659817'

In [114]:
# Create a new loan
if index != -1:
    loanieType = accountsConract.functions.getType(index).call()
    if  loanieType:
        if createLoan(_loanie=web3.eth.accounts[2], _loaner=account, _amount=random.randint(1,100000), _organizationContract=organizationContract):
            print("Loan created")
else:
    print("This account is not registered in our system.")

Enter the password of the organization: ········
SignedTransaction(rawTransaction=HexBytes('0xf8ab01843b9aca008301117094ca4e05ae40d86ab70ce485501c17100c0340067a80b8441dd6f23c000000000000000000000000ee4f5078d802207d68ee9668ae6b82fcfcf7e2900000000000000000000000000000000000000000000000000000000000002f92820a95a0a291cd3a9afbec50ef048c512b9f56f31a3989e42e5cd1d63c4749f47c3ec7daa060466a27ed9dad80381d38e5b4964e7d57c1a6a69e791e9063ff6d9d74f41eac'), hash=HexBytes('0x7f5ca699c43bd4659744c396492be101b30f3fddc590bab3a7825b6af57125dc'), r=73532290732881747477918619258326501325909084437364388580309469658135192848346, s=43546445420102617989995874739404894984600188526647052220838945642816570924716, v=2709)


ValueError: {'message': 'VM Exception while processing transaction: revert', 'code': -32000, 'data': {'0x7f5ca699c43bd4659744c396492be101b30f3fddc590bab3a7825b6af57125dc': {'error': 'revert', 'program_counter': 568, 'return': '0x'}, 'stack': 'RuntimeError: VM Exception while processing transaction: revert\n    at Function.RuntimeError.fromResults (/opt/ganache/resources/static/node/node_modules/ganache-core/lib/utils/runtimeerror.js:89:13)\n    at BlockchainDouble.processBlock (/opt/ganache/resources/static/node/node_modules/ganache-core/lib/blockchain_double.js:632:24)\n    at runMicrotasks (<anonymous>)\n    at processTicksAndRejections (internal/process/task_queues.js:93:5)', 'name': 'RuntimeError'}}